In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
data = pd.read_excel(r"insta_data_sample.xlsx")
data

,no,date,datehour,datetime,caption,like_cnt,comment_cnt
0,5,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:31:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,17,0
1,13,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,9,0
2,12,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,6,0
3,3,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,6,0
4,14,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:28:00,#먹스타그램 #먹스타 #맛스타그램 #맛스타 #디저트 #케이크 #디저트그램 #서촌 #...,7,0
...,...,...,...,...,...,...,...
11356,66011,2014-10-30,2014-10-30 17:00:00,2014-10-30 17:59:00,오늘의 메뉴.\n\n제주도야지로만든 제육덮밥.\n\n저녁시간이니 서두르세요~~ 맛있...,96,10
11357,66026,2014-10-18,2014-10-18 22:00:00,2014-10-18 22:42:00,@regokim 미리생축하여 @hudiniapa 가쏜다~~!! 잘먹겠습니다!! #족...,88,0
11358,66061,2014-10-06,2014-10-06 18:00:00,2014-10-06 18:44:00,145528,53,4
11359,66099,2014-09-04,2014-09-04 20:00:00,2014-09-04 20:57:00,너무 맛있게 먹었던\n #오징어먹물리조또 <f0><U+009F><U+0091><U+...,67,3


In [47]:
# 우선 전부 1값으로 주기
data['label'] = 1
data.head()

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
0,5,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:31:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,17,0,1
1,13,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,9,0,1
2,12,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,6,0,1
3,3,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,6,0,1
4,14,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:28:00,#먹스타그램 #먹스타 #맛스타그램 #맛스타 #디저트 #케이크 #디저트그램 #서촌 #...,7,0,1


In [48]:
# '#'이 포함되지 않은 데이터들 0(라벨값 이하 부르지않음) 값으로 변경
i=0
for i in range(len(data)) : 
    data.at[i,'caption'] = str(data.at[i,'caption'])
    if data.at[i,'caption'].find('#') == -1:
        data.at[i,'label'] = 0

In [5]:
data[data['label'] == 0].head()

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
8,25,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:20:00,이니셜 각인으로 나만의 케이스를 만들어 보세요!\n배경색 및 글씨색 변경 가능 <f...,8,2,0
14,33,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:14:00,<f0><U+009F><U+0092><U+0093><f0><U+009F><U+009...,15,3,0
15,34,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:13:00,샐러드 먹구 만칼로리 디져트 먹기 <f0><U+009F><U+008D><U+00AA...,16,3,0
17,36,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:12:00,.\n해독은 다이어트 방법이 아닙니다. \n그동안 쌓아둔 내 몸의 독소를 빼주는 건...,8,1,0
19,39,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:11:00,Haven't got school today<f0><U+009F><U+0092><U...,42,3,0


In [415]:
data[data['label'] == 0]

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
8,25,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:20:00,이니셜 각인으로 나만의 케이스를 만들어 보세요!_x000D_\n배경색 및 글씨색 변...,8,2,0
14,33,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:14:00,<f0><U+009F><U+0092><U+0093><f0><U+009F><U+009...,15,3,0
15,34,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:13:00,샐러드 먹구 만칼로리 디져트 먹기 <f0><U+009F><U+008D><U+00AA...,16,3,0
17,36,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:12:00,._x000D_\n해독은 다이어트 방법이 아닙니다. _x000D_\n그동안 쌓아둔 ...,8,1,0
19,39,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:11:00,Haven't got school today<f0><U+009F><U+0092><U...,42,3,0
...,...,...,...,...,...,...,...,...
8999,54239,2017-08-17,2017-08-17 13:00:00,2017-08-17 13:02:00,-_x000D_\n꾸준한 인기의 악어패턴 케이스 <f0><U+009F><U+0092...,21,3,0
9016,54349,2017-08-16,2017-08-16 10:00:00,2017-08-16 10:38:00,-_x000D_\n골드라인의 심플한디자인의 케이스<U+3030> 펄컬러<U+2728...,43,0,0
9033,54435,2017-08-14,2017-08-14 20:00:00,2017-08-14 20:27:00,-_x000D_\n커플케이스로 딱인 고야드 시리즈<f0><U+009F><U+0096...,16,1,0
9153,55134,2017-07-30,2017-07-30 14:00:00,2017-07-30 14:34:00,-_x000D_\n가죽느낌의 핑쿠핑쿠한 장미자수케이스<f0><U+009F><U+00...,46,0,0


In [416]:
data.head()

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
0,5,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:31:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,17,0,1
1,13,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,9,0,1
2,12,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,6,0,1
3,3,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,#이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...,6,0,1
4,14,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:28:00,#먹스타그램 #먹스타 #맛스타그램 #맛스타 #디저트 #케이크 #디저트그램 #서촌 #...,7,0,1


In [49]:
# '상담',  '문의', '전화', '프로필링크 클릭', '세트', '택배', '플러스친구',
#'다이어트', '디톡스', '요요', '젤네일', '속눈썹연장','하나님'
# 내용 포함된 것 0으로 변경
i=0
pattern = ['상담',  '문의', '전화', '프로필링크 클릭',\
                 '세트', '택배', '플러스친구','다이어트', '디톡스', \
                 '요요', '젤네일', '속눈썹연장','하나님']
for i in range(len(data)) : 
    if any(pat in data.at[i,'caption'] for pat in pattern) :
        data.at[i,'label'] = 0
        

In [418]:
data[data['label'] == 0].head()

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
6,21,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:24:00,이태원 blue55 _x000D_\n#와인 #wine #파티 #포토_x000D_\n...,24,0,0
8,25,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:20:00,이니셜 각인으로 나만의 케이스를 만들어 보세요!_x000D_\n배경색 및 글씨색 변...,8,2,0
10,24,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:20:00,메탈파우더랑 스와만남<U+2728><U+2728><U+2728><U+2728><U+...,7,1,0
13,32,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:14:00,"직접 체험해보기 전엔 탈모케어에 자신이 없었는데, 한달 케어해보니 자신감 상승<U+...",22,0,0
14,33,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:14:00,<f0><U+009F><U+0092><U+0093><f0><U+009F><U+009...,15,3,0


In [13]:
len(data[data['label'] == 0])

1549

In [50]:
# 해당 지역에 맛집 붙여서, 그 단어가 존재하면 0으로 바꾸기
i=0
pattern1 = ['홍대', '강남', '부산', '광주', '건대', \
           '마포', '공덕', '송정역', '송정', '공항동', '공항', '마곡동', \
           '마곡', '숙대', '압구정', '광화문', '안산']

pattern1_mj = [x+'맛집' for x in pattern1]

for i in range(len(data)) : 
    
    if any(pat in data.at[i,'caption'] for pat in pattern1) :
         data.at[i,'label'] = 0
#         print(data.at[i,'caption'])
    if any(pat in data.at[i,'caption'] for pat in pattern1_mj) :
        data.at[i,'label'] = 0

In [41]:
len(data[data['label'] == 0]) # 맛집만

3282

In [46]:
len(data[data['label'] == 0]) # 지역만

3425

In [51]:
len(data[data['label'] == 0]) #다 합쳐서

3425

# 넣지 않은 다른 지역을 넣기
# 지역 주변이 이태원 주변 어딨는지 확인
# 지역이름만 넣기

In [421]:
pattern

['상담',
 '문의',
 '전화',
 '프로필링크 클릭',
 '세트',
 '택배',
 '플러스친구',
 '다이어트',
 '디톡스',
 '요요',
 '젤네일',
 '속눈썹연장',
 '하나님']

In [422]:
for x in pattern1 : 
    print(x)
for x in pattern1_mj : 
    print(x)

홍대
강남
부산
광주
건대
마포
공덕
송정역
송정
공항동
공항
마곡동
마곡
숙대
압구정
광화문
안산
홍대맛집
강남맛집
부산맛집
광주맛집
건대맛집
마포맛집
공덕맛집
송정역맛집
송정맛집
공항동맛집
공항맛집
마곡동맛집
마곡맛집
숙대맛집
압구정맛집
광화문맛집
안산맛집


In [423]:
data[data['label'] == 0].head()

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
4,14,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:28:00,#먹스타그램 #먹스타 #맛스타그램 #맛스타 #디저트 #케이크 #디저트그램 #서촌 #...,7,0,0
6,21,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:24:00,이태원 blue55 _x000D_\n#와인 #wine #파티 #포토_x000D_\n...,24,0,0
8,25,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:20:00,이니셜 각인으로 나만의 케이스를 만들어 보세요!_x000D_\n배경색 및 글씨색 변...,8,2,0
10,24,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:20:00,메탈파우더랑 스와만남<U+2728><U+2728><U+2728><U+2728><U+...,7,1,0
13,32,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:14:00,"직접 체험해보기 전엔 탈모케어에 자신이 없었는데, 한달 케어해보니 자신감 상승<U+...",22,0,0


In [424]:
len(data[data['label'] == 0])

3282

In [52]:
# 선정적인 글 라벨 0으로 바꾸기
i=0
pattern2 = ['비글녀']

for i in range(len(data)) : 
    if any(pat in data.at[i,'caption'] for pat in pattern2) :
        data.at[i,'label'] = 0
#         print(data.at[i,'caption'])

In [53]:
len(data[data['label'] == 0])

3425

# 데이터 전처리

In [427]:
data['label'].value_counts(normalize=True)

1    0.710853
0    0.289147
Name: label, dtype: float64

In [428]:
data.caption

0        #이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...
1        #이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...
2        #이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...
3        #이태원맛집#이태원술집#와인바#경리단길#녹사평맛집#경리단길술집#경리단길맛집#이태원#...
4        #먹스타그램 #먹스타 #맛스타그램 #맛스타 #디저트 #케이크 #디저트그램 #서촌 #...
                               ...                        
11356    오늘의 메뉴._x000D_\n_x000D_\n제주도야지로만든 제육덮밥._x000D_...
11357    @regokim 미리생축하여 @hudiniapa 가쏜다~~!! 잘먹겠습니다!! #족...
11358                                               145528
11359    너무 맛있게 먹었던_x000D_\n #오징어먹물리조또 <f0><U+009F><U+0...
11360    #대파피자 #보석길 #경리단 #칼솟타다 _x000D_\n#이태원 #이태원맛집 #피자...
Name: caption, Length: 11361, dtype: object

In [429]:
# '#' 없애기
for i in range(len(data)) : 
    data.at[i,'caption'] = re.sub(pattern = '#', repl = ' ', string = data.at[i,'caption'])
    
data

,no,date,datehour,datetime,caption,like_cnt,comment_cnt,label
0,5,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:31:00,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,17,0,1
1,13,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,9,0,1
2,12,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,6,0,1
3,3,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:29:00,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,6,0,1
4,14,2017-12-06,2017-12-06 17:00:00,2017-12-06 17:28:00,먹스타그램 먹스타 맛스타그램 맛스타 디저트 케이크 디저트그램 서촌 ...,7,0,0
...,...,...,...,...,...,...,...,...
11356,66011,2014-10-30,2014-10-30 17:00:00,2014-10-30 17:59:00,오늘의 메뉴._x000D_\n_x000D_\n제주도야지로만든 제육덮밥._x000D_...,96,10,1
11357,66026,2014-10-18,2014-10-18 22:00:00,2014-10-18 22:42:00,@regokim 미리생축하여 @hudiniapa 가쏜다~~!! 잘먹겠습니다!! 족...,88,0,1
11358,66061,2014-10-06,2014-10-06 18:00:00,2014-10-06 18:44:00,145528,53,4,0
11359,66099,2014-09-04,2014-09-04 20:00:00,2014-09-04 20:57:00,너무 맛있게 먹었던_x000D_\n 오징어먹물리조또 <f0><U+009F><U+0...,67,3,1


In [430]:
# 이모티콘 없애기
# pattern4 = ['<U+','<f','_x','000']
re.compile("(<U+)")
for i in range(len(data)) : 
    data.at[i,'caption'] = re.sub('<[\w+]+>|<f0>|_x+[\w+]+_|\n|', '', data.at[i,'caption'])

In [431]:
# data.at[11359,'caption']
'''
'너무 맛있게 먹었던_x000D_\n  오징어먹물리조또 <f0><U+009F><U+0091><U+008D><f0><U+009F><U+0091><U+008F> 후기 <f0><U+009F><U+0091><U+0089>_x000D_\nhttp://blog.naver.com/soheelook/220113468782_x000D_\n_x000D_\n 장진우의그랑블루 _x000D_\n 먹스타그램  맛스타그램  먹방  오징어  리조또  장진우  그랑블루  푸드스타그램  foodstagram  이태원  이태원맛집  보석길  회나무길  이태원프리덤 ㅋㄷ'
'''

"\n'너무 맛있게 먹었던_x000D_\n  오징어먹물리조또 <f0><U+009F><U+0091><U+008D><f0><U+009F><U+0091><U+008F> 후기 <f0><U+009F><U+0091><U+0089>_x000D_\nhttp://blog.naver.com/soheelook/220113468782_x000D_\n_x000D_\n 장진우의그랑블루 _x000D_\n 먹스타그램  맛스타그램  먹방  오징어  리조또  장진우  그랑블루  푸드스타그램  foodstagram  이태원  이태원맛집  보석길  회나무길  이태원프리덤 ㅋㄷ'\n"

In [432]:
# sdata = data.at[11359,'caption']

In [433]:
# re.sub('<[\w+]+>|<f0>|_x+[\w+]+_|\n|', '', sdata)
'''
'너무 맛있게 먹었던  오징어먹물리조또  후기 http://blog.naver.com/soheelook/220113468782 장진우의그랑블루  먹스타그램  맛스타그램  먹방  오징어  리조또  장진우  그랑블루  푸드스타그램  foodstagram  이태원  이태원맛집  보석길  회나무길  이태원프리덤 ㅋㄷ'
'''

"\n'너무 맛있게 먹었던  오징어먹물리조또  후기 http://blog.naver.com/soheelook/220113468782 장진우의그랑블루  먹스타그램  맛스타그램  먹방  오징어  리조또  장진우  그랑블루  푸드스타그램  foodstagram  이태원  이태원맛집  보석길  회나무길  이태원프리덤 ㅋㄷ'\n"

In [434]:
data.caption

0         이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...
1         이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...
2         이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...
3         이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...
4         먹스타그램  먹스타  맛스타그램  맛스타  디저트  케이크  디저트그램  서촌  ...
                               ...                        
11356    오늘의 메뉴.제주도야지로만든 제육덮밥.저녁시간이니 서두르세요~~ 맛있게 야무지게 드...
11357    @regokim 미리생축하여 @hudiniapa 가쏜다~~!! 잘먹겠습니다!!  족...
11358                                               145528
11359    너무 맛있게 먹었던  오징어먹물리조또  후기 http://blog.naver.com...
11360      대파피자  보석길  경리단  칼솟타다  이태원  이태원맛집  피자  대파  먹스타그램
Name: caption, Length: 11361, dtype: object

In [435]:
# 특수기호처리

# 데이터 분리

In [436]:
using_data = data[['caption', 'label']]
using_data

,caption,label
0,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,1
1,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,1
2,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,1
3,이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 ...,1
4,먹스타그램 먹스타 맛스타그램 맛스타 디저트 케이크 디저트그램 서촌 ...,0
...,...,...
11356,오늘의 메뉴.제주도야지로만든 제육덮밥.저녁시간이니 서두르세요~~ 맛있게 야무지게 드...,1
11357,@regokim 미리생축하여 @hudiniapa 가쏜다~~!! 잘먹겠습니다!! 족...,1
11358,145528,0
11359,너무 맛있게 먹었던 오징어먹물리조또 후기 http://blog.naver.com...,1


In [437]:
using_data['caption'][0]

' 이태원맛집 이태원술집 와인바 경리단길 녹사평맛집 경리단길술집 경리단길맛집 이태원 녹사평술집 루프탑 테라스 먹스타그램 분위기깡패 와인바 망년회 퐁듀 새우맛집 연말모임 모임 회식 데이트 감성자극 저렴한와인바 취향저격 논현맛집 이태원핫플레이스 연어맛집 벌집퐁듀'

In [438]:
x = using_data.caption
y = using_data.label

In [439]:
from sklearn.model_selection import train_test_split

In [440]:
train_X, test_X, train_Y, test_Y = train_test_split(x,y,test_size=0.2,random_state=0)

In [441]:
train_X.shape, train_Y.shape

((9088,), (9088,))

In [442]:
test_X.shape, test_Y.shape

((2273,), (2273,))

In [443]:
train_X

4941     .브루스게타 5피스(Bruschetta) 계절마다 조금씩 바뀌는 푼토나인의 브루스게...
1411     December  ...... travelphotographer  stylelife...
1835     {  코레아노스키친  먹스타그램  맛스타그램  맛있다그램  맛점  점심  부리또  ...
10321    Thanks~~~ 26ontap whiterabbit whiterabbittapho...
2113     @arianagrande ~@grandexchill~  후식  커피  좋아요  이태...
                               ...                        
4859     입구 벤허제작~ 인스타처럼 아이디어를 내봤는데..어떨까욤??  롱아일랜드펍  셀카 ...
3264     BAEEEE Follow @onikafeens for more  NickiMinaj...
9845      동구다방 카페라떼와  다르시 장미.... 이태원카페 한남동카페 우사단길 우사단길카...
10799    비쥬얼은 굿.맛은.... . 오늘의메뉴  먹스타그램  먹방  맛스타그램  맛집  맛...
2732                                                 13307
Name: caption, Length: 9088, dtype: object

# 머신 돌리기

## 로지스틱회귀분류모델

In [444]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()
vect.fit(train_X)
train_X_vect = vect.transform(train_X)
test_X_vect = vect.transform(test_X)
train_X_vect.shape

(9088, 50608)

In [445]:
vect.get_feature_names()

C:\Users\medici\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '000개를',
 '000명',
 '000원',
 '000원2세트',
 '000원5세트',
 '000원부터',
 '000원이라는',
 '000원인건',
 '000원정보',
 '000원팔로워',
 '001도',
 '0092',
 '0099',
 '00am',
 '00am가격',
 '00am토',
 '00b',
 '00break',
 '00everyday',
 '00if',
 '00last',
 '00p',
 '00pm',
 '00금토일',
 '00까지',
 '00상황에',
 '00지하철',
 '01',
 '010',
 '010004',
 '0100058',
 '0100066',
 '010026',
 '0101374',
 '0101637',
 '0101644',
 '0101854',
 '0101894',
 '0102698',
 '0102819',
 '0102828',
 '0102831',
 '0102859',
 '0102863',
 '0103109',
 '0103114',
 '01042006938비원라운지클럽',
 '0104202',
 '0104266',
 '0104271',
 '0104306',
 '0104307',
 '01044066246단체주문',
 '0104871',
 '0104965',
 '0105086',
 '0105678',
 '0105680',
 '0107123',
 '0107363',
 '0107586',
 '0107999',
 '0108037',
 '0108149',
 '0108825',
 '0108826',
 '0108888',
 '0108935',
 '0108937',
 '0108992',
 '0109078',
 '0109080',
 '0110115',
 '0110116',
 '0110133',
 '0110228',
 '0110235',
 '0112185',
 '0113511',
 '0114687',
 '0114688',
 '0114689',
 '0114690',
 '0114967',
 '011497',
 '011

In [446]:
vect.vocabulary_

{'브루스게타': 28711,
 '5피스': 2473,
 'bruschetta': 4330,
 '계절마다': 15725,
 '조금씩': 41880,
 '바뀌는': 26502,
 '푼토나인의': 47474,
 '브루스게타입니다바게트빵은': 28712,
 '오월의종': 36809,
 '쓰고있습니다': 33392,
 '푼토나인': 47473,
 '영업시간11': 36176,
 '30': 1953,
 '00break': 19,
 'time5': 12328,
 '10': 422,
 '30last': 1972,
 'order': 9884,
 '30pm': 1975,
 '예약문의': 36340,
 '02': 155,
 '790': 2635,
 '9545': 2769,
 'puntonine': 10616,
 'punto9': 10615,
 '푼토9': 47472,
 '타파스': 46138,
 'tapas': 12061,
 '브런치': 28656,
 'brunch': 4325,
 'december': 5515,
 'travelphotographer': 12464,
 'stylelife': 11852,
 'outfitoftheday': 9933,
 'lovephoto': 8704,
 'dreamingbig': 5845,
 'justlife': 8057,
 'live': 8594,
 'photography': 10246,
 'photoshoot': 10251,
 'young': 13266,
 'teenager': 12129,
 'travel': 12459,
 'artofvisuals': 3396,
 'travelphotography': 12465,
 'livinginspain': 8600,
 'huntgram': 7482,
 'visualoflife': 12847,
 'iger': 7560,
 'unknownspain': 12634,
 '먹스타그램': 24896,
 '셀스타그램': 31040,
 '맛스타그램': 23974,
 '맛점': 24256,
 '점심': 41442,
 '타

In [447]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [448]:
lr = LogisticRegression()
lr.fit(train_X_vect,train_Y)
pred_Y = lr.predict(test_X_vect)
accuracy_score(test_Y, pred_Y)

0.9511658600967884

# 키워드 별로 분석해보기
정답인 것들 중에 

해방촌이란 키워드가 들어갔을 때, 무조건 정답이라고 인식하면 그 해방촌이랑 키워드를 어떻게 처리해야할까?

키워드 별로 얼마나 나왔는지, 라벨을 결정하는데 얼마나 영향력을 미쳤는지, 해당 키워드가 들어갔을 때 재현율, 정밀도는 얼마나 나왔는지